## Dataset Creation

In [1]:
from torch.utils.data import Dataset
from glob import glob
import torch
from torch import nn
from PIL import Image
from torchvision.transforms.functional import resize, pil_to_tensor

class GenderDataset(Dataset):
    def __init__(self, data = "./Gender_Detection_Tiny", split='Train', size=64):
        self.data = glob(data + '/' + split + '/Male/*.jpg') + glob(data + '/' + split + '/Female/*.jpg')
        print(len(self.data))
        self.labels = [1 if 'Male' in x else 0 for x in self.data]
        self.image_size = size

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        image = Image.open(self.data[index])
        image = pil_to_tensor(image)
        image = resize(image, (self.image_size, self.image_size), antialias=True)
        image = image / 127.5 - 1

        return image, self.labels[index]

celeb_data = GenderDataset(data = "./Gender_Detection_Tiny", split='Train', size=64)
celeb_dataloder = torch.utils.data.DataLoader(celeb_data, batch_size=32, shuffle=True)

18000


## Model Creation

In [2]:
from torch import nn
import torch

class BasicBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, 
                               stride=stride, padding=1, bias=False)
        num_groups = min(8, out_channels) if out_channels >= 8 else out_channels
        self.bn1 = nn.GroupNorm(num_groups, out_channels)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn2 = nn.GroupNorm(num_groups, out_channels)
        
        # Shortcut connection
        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1,
                         stride=stride, bias=False),
                nn.GroupNorm(num_groups, out_channels)
            )
    
    def forward(self, x):
        identity = x
        out = self.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out = out + self.shortcut(identity)
        out = self.relu(out)
        return out

# ResNet for MNIST
class ResNet(nn.Module):
    def __init__(self, num_classes=10):
        super(ResNet, self).__init__()
        self.in_channels = 16
        
        # Initial convolution (adjusted for 28x28 MNIST)
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.GroupNorm(8, 16)
        self.relu = nn.ReLU()
        
        # ResNet layers
        self.layer1 = self._make_layer(16, 2, stride=1)
        self.layer2 = self._make_layer(32, 2, stride=2)
        self.layer3 = self._make_layer(64, 2, stride=2)
        
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(64, num_classes)
    
    def _make_layer(self, out_channels, num_blocks, stride):
        layers = []
        layers.append(BasicBlock(self.in_channels, out_channels, stride))
        self.in_channels = out_channels
        for _ in range(1, num_blocks):
            layers.append(BasicBlock(out_channels, out_channels, stride=1))
        return nn.Sequential(*layers)
    
    def forward(self, x):
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x

# Create an instance of ResNetMNIST
model = ResNet(num_classes=2)

## Pre-Training

In [3]:
from tqdm import tqdm

def accuracy_score(y_true, y_pred):
    train_acc = torch.sum(y_pred == y_true).item() / len(y_true)
    return train_acc

# Train loop and save the model
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
prev_best_acc = 0
print("Starting SGD training...")
print("=" * 50)
device = torch.device('mps')
model = model.to(device)

# Training loop with DP-SGD
num_epochs = 5
for epoch in range(num_epochs):
    running_loss = 0.0
    running_acc = 0.0
    num_batches = 0
        
    for inputs, labels in tqdm(celeb_dataloder, desc=f"Epoch {epoch + 1}/{num_epochs}"):
        # Move to device if using GPU
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        # Zero the parameter gradients
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(inputs)
        
        # Compute loss
        loss = criterion(outputs, labels)
        
        # Backward pass
        loss.backward()
        
        # Optimizer step (with DP noise added)
        optimizer.step()
        
        # Accumulate statistics
        running_loss += loss.item()
        
        # Compute PSNR for this batch
        with torch.no_grad():
            acc = accuracy_score(labels.cpu().detach(), outputs.argmax(1).cpu().detach())
            running_acc += acc
                
        num_batches += 1
    
    # Print epoch statistics
    avg_loss = running_loss / num_batches
    avg_acc = running_acc / num_batches
    
    print(f'Epoch {epoch + 1}/{num_epochs}')
    print(f'  Loss: {avg_loss:.4f}')
    print(f'  Accuracy: {avg_acc:.4f}')
    print('-' * 50)
    
    # Save the checkpoint
    import os
    os.makedirs('checkpoints_celeb', exist_ok=True)
    torch.save(model.state_dict(), f'checkpoints_celeb/model_epoch_{epoch + 1}.pth')
    if avg_acc > prev_best_acc:
        # save the best model yet
        prev_best_acc = avg_acc
        torch.save(model.state_dict(), f'checkpoints_celeb/model_best.pth')

Starting SGD training...


Epoch 1/5: 100%|██████████| 563/563 [01:53<00:00,  4.97it/s]


Epoch 1/5
  Loss: 0.6030
  Accuracy: 0.6642
--------------------------------------------------


Epoch 2/5: 100%|██████████| 563/563 [01:50<00:00,  5.11it/s]


Epoch 2/5
  Loss: 0.4292
  Accuracy: 0.8005
--------------------------------------------------


...

Epoch 3/5
  Loss: 0.3351
  Accuracy: 0.8545
--------------------------------------------------


Epoch 4/5: 100%|██████████| 563/563 [01:50<00:00,  5.09it/s]


Epoch 4/5
  Loss: 0.2892
  Accuracy: 0.8770
--------------------------------------------------


Epoch 5/5: 100%|██████████| 563/563 [01:50<00:00,  5.07it/s]

Epoch 5/5
  Loss: 0.2518
  Accuracy: 0.8968
--------------------------------------------------


## Dataset preperation for finetuning

In [4]:
from torch.utils.data import Dataset
from glob import glob
import torch
from torch import nn
from PIL import Image
from torchvision.transforms.functional import resize, pil_to_tensor

class GenderDataset(Dataset):
    def __init__(self, data = "./Gender Detection Tiny", split='Train', size=64):
        self.data = glob(data + '/' + split + '/Male/*.jpg') + glob(data + '/' + split + '/Female/*.jpg')
        print(len(self.data))
        self.labels = [1 if 'Male' in x else 0 for x in self.data]
        self.image_size = size

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        image = Image.open(self.data[index])
        image = pil_to_tensor(image)
        image = resize(image, (self.image_size, self.image_size), antialias=True)
        image = image / 127.5 - 1

        return image, self.labels[index]

celeb_data = GenderDataset(data = "./Gender_Detection_Tiny", split='Test', size=64)
celeb_dataloder = torch.utils.data.DataLoader(celeb_data, batch_size=32, shuffle=True)

1001


## Global Private-Finetuning

In [31]:
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm
from opacus import PrivacyEngine

def accuracy_score(y_true, y_pred):
    train_acc = torch.sum(y_pred == y_true).item() / len(y_true)
    return train_acc

# Create the fixed model
print("Creating Opacus-compatible model")
model = ResNet(num_classes=2)
model = model.to(device)
# If you want to copy weights from your existing model:
model_path = "./checkpoints_celeb/model_best.pth"
model.load_state_dict(torch.load(model_path))

# ---------------------------
# DP-SGD Training
# ---------------------------
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.00001, momentum=0.9)

# Initialize Privacy Engine
privacy_engine = PrivacyEngine(secure_mode=False)  # Set to True for production

# Make the model, optimizer, and data loader private
model, optimizer, celeb_dataloderloader = privacy_engine.make_private(
    module=model,
    optimizer=optimizer,
    data_loader=celeb_dataloder,
    noise_multiplier=2.0,  # Controls privacy-utility tradeoff (higher = more private)
    max_grad_norm=0.000001,     # Gradient clipping threshold
)

print("Starting DP-SGD training...")
print("=" * 50)
prev_best_acc = 0

# Training loop with DP-SGD
num_epochs = 5
for epoch in range(num_epochs):
    running_loss = 0.0
    running_acc = 0.0
    num_batches = 0
        
    for inputs, labels in tqdm(celeb_dataloder, desc=f"Epoch {epoch + 1}/{num_epochs}"):
        # Move to device if using GPU
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        # Zero the parameter gradients
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(inputs)
        
        # Compute 
        loss = criterion(outputs, labels)
        
        # Backward pass
        loss.backward()
        
        # Optimizer step (with DP noise added)
        optimizer.step()
        
        # Accumulate statistics
        running_loss += loss.item()
        
        # Compute accuracy for this batch
        with torch.no_grad():
            acc = accuracy_score(labels.cpu().detach(), outputs.argmax(1).cpu().detach())
            running_acc += acc
        
        num_batches += 1
    
    # Print epoch statistics
    avg_loss = running_loss / num_batches
    avg_acc = running_acc / num_batches
    
    print(f'Epoch {epoch + 1}/{num_epochs}')
    print(f'  Loss: {avg_loss:.4f}')
    print(f'  Accuracy: {avg_acc:.4f}')
    
    # Save the checkpoint
    import os
    os.makedirs('checkpoints/privacy', exist_ok=True)
    torch.save(model.state_dict(), f'checkpoints/privacy/model_epoch_{epoch + 1}.pth')
    if avg_acc > prev_best_acc:
        # save the best model yet
        prev_best_acc = avg_acc
        torch.save(model.state_dict(), f'checkpoints/privacy/model_best.pth')

Creating Opacus-compatible model
Starting DP-SGD training...


Epoch 1/5: 100%|██████████| 32/32 [00:16<00:00,  1.97it/s]


Epoch 1/5
  Loss: 0.2274
  Accuracy: 0.8956


Epoch 2/5: 100%|██████████| 32/32 [00:16<00:00,  1.93it/s]


Epoch 2/5
  Loss: 0.2463
  Accuracy: 0.8991


...

Epoch 3/5
  Loss: 0.2540
  Accuracy: 0.8928


Epoch 4/5: 100%|██████████| 32/32 [00:15<00:00,  2.08it/s]


Epoch 4/5
  Loss: 0.2646
  Accuracy: 0.8878


Epoch 5/5: 100%|██████████| 32/32 [00:15<00:00,  2.03it/s]

Epoch 5/5
  Loss: 0.2505
  Accuracy: 0.8940


## Local Private-Finetuning

In [30]:
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm
from opacus import PrivacyEngine

def accuracy_score(y_true, y_pred):
    train_acc = torch.sum(y_pred == y_true).item() / len(y_true)
    return train_acc

# Create the fixed model
print("Creating Opacus-compatible model")
model = ResNet(num_classes=2)
model = model.to(device)
# If you want to copy weights from your existing model:
model_path = "./checkpoints_celeb/model_best.pth"
model.load_state_dict(torch.load(model_path))

# ---------------------------
# DP-SGD Training
# ---------------------------
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.00001, momentum=0.9)

# Initialize Privacy Engine
privacy_engine = PrivacyEngine(secure_mode=False)  # Set to True for production

# Make the model, optimizer, and data loader private
model, optimizer, celeb_dataloder = privacy_engine.make_private(
    module=model,
    optimizer=optimizer,
    data_loader=celeb_dataloder,
    noise_multiplier=1.0,  # Controls privacy-utility tradeoff (higher = more private)
    max_grad_norm=0.00001,     # Gradient clipping threshold
)

print("Starting DP-SGD training...")
print("=" * 50)
prev_best_acc = 0

# Training loop with DP-SGD
num_epochs = 5
for epoch in range(num_epochs):
    running_loss = 0.0
    running_acc = 0.0
    num_batches = 0
        
    for inputs, labels in tqdm(celeb_dataloder, desc=f"Epoch {epoch + 1}/{num_epochs}"):
        # Move to device if using GPU
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        # Zero the parameter gradients
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(inputs)
        
        # Compute 
        loss = criterion(outputs, labels)
        
        # Backward pass
        loss.backward()
        
        # Optimizer step (with DP noise added)
        optimizer.step()
        
        # Accumulate statistics
        running_loss += loss.item()
        
        # Compute accuracy for this batch
        with torch.no_grad():
            acc = accuracy_score(labels.cpu().detach(), outputs.argmax(1).cpu().detach())
            running_acc += acc
        
        num_batches += 1
    
    # Print epoch statistics
    avg_loss = running_loss / num_batches
    avg_acc = running_acc / num_batches
    
    print(f'Epoch {epoch + 1}/{num_epochs}')
    print(f'  Loss: {avg_loss:.4f}')
    print(f'  Accuracy: {avg_acc:.4f}')
    
    # Save the checkpoint
    import os
    # os.makedirs('checkpoints/privacy', exist_ok=True)
    # torch.save(model.state_dict(), f'checkpoints/privacy/model_epoch_{epoch + 1}.pth')
    # if avg_acc > prev_best_acc:
    #     # save the best model yet
    #     prev_best_acc = avg_acc
    #     torch.save(model.state_dict(), f'checkpoints/privacy/model_best.pth')

Creating Opacus-compatible model
Starting DP-SGD training...


Epoch 1/5: 100%|██████████| 32/32 [00:19<00:00,  1.67it/s]


Epoch 1/5
  Loss: 0.2220
  Accuracy: 0.9114


Epoch 2/5: 100%|██████████| 32/32 [00:14<00:00,  2.15it/s]


Epoch 2/5
  Loss: 0.2260
  Accuracy: 0.9090


...

Epoch 3/5
  Loss: 0.2776
  Accuracy: 0.8841


Epoch 4/5: 100%|██████████| 32/32 [00:14<00:00,  2.15it/s]


Epoch 4/5
  Loss: 0.2341
  Accuracy: 0.9006


Epoch 5/5: 100%|██████████| 32/32 [00:14<00:00,  2.15it/s]

Epoch 5/5
  Loss: 0.2581
  Accuracy: 0.8926
